# 문장 요약 학습2

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
import torch.nn as nn

In [ ]:
# 데이터 로딩 및 전처리

import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv('/content/example_data.csv')
df_train, df_test = train_test_split(df, test_size=0.2)
df_train, df_val = train_test_split(df_train, test_size=0.25)


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.1 MB/s eta 0:00:00


In [ ]:
# 토큰화

from transformers import PegasusTokenizer

tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

def tokenize_data(df):
    inputs = tokenizer(df['가사 전문'].tolist(), padding='max_length', truncation=True, max_length=512)
    targets = tokenizer(df['가사 요약'].tolist(), padding='max_length', truncation=True, max_length=128)
    return inputs, targets

train_inputs, train_targets = tokenize_data(df_train)
val_inputs, val_targets = tokenize_data(df_val)
test_inputs, test_targets = tokenize_data(df_test)


ImportError: ignored

In [ ]:
# 모델 학습

from transformers import PegasusForConditionalGeneration
import torch
from torch.utils.data import DataLoader, TensorDataset

train_data = TensorDataset(torch.tensor(train_inputs['input_ids']), torch.tensor(train_targets['input_ids']))
train_loader = DataLoader(train_data, batch_size=2)

# 모델 초기화
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 최적화 함수 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 손실 함수 설정 (Pegasus에 내장된 손실 함수 사용)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# 학습 루프
num_epochs = 3  # 에포크 수 설정
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        # 입력과 타겟을 디바이스로 이동
        input_ids = batch[0].to(device)
        labels = batch[1].to(device)

        # 모델의 출력 계산
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")


In [ ]:
!pip install rouge_score

In [ ]:
# 모델 평가

from datasets import load_metric

# 평가 준비: DataLoader 생성
val_data = TensorDataset(torch.tensor(val_inputs['input_ids']), torch.tensor(val_targets['input_ids']))
val_loader = DataLoader(val_data, batch_size=2)  # 배치 크기 조정 가능

# ROUGE 점수를 계산하기 위한 메트릭 로드
rouge = load_metric('rouge')

# 평가 루프
model.eval()  # 모델을 평가 모드로 설정
total_loss = 0
for batch in val_loader:
    input_ids = batch[0].to(device)
    labels = batch[1].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # 요약 생성 및 ROUGE 점수 계산
        predictions = model.generate(input_ids)
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        rouge.compute(predictions=decoded_preds, references=decoded_labels)

# 평균 손실 계산
avg_loss = total_loss / len(val_loader)
print(f'Validation Loss: {avg_loss}')



In [ ]:
# 모델을 사용하여 요약 생성

input_text = """
술마시고 노래하고 춤을 춰봐도
가슴에는 하나 가득 슬픔뿐이네
무엇을 할 것인가 둘러 보아도
보이는 건 모두가 돌아 앉았네
자 떠나자 동해 바다로
삼등삼등 완행열차 기차를 타고
간밤에 꾸었던 꿈의 세계는
아침에 일어나면 잊혀지지만
그래도 생각나는 내 꿈 하나는
조그만 예쁜 고래 한마리
자 떠나자 동해 바다로
신화처럼 숨을 쉬는 고래 잡으러
우리의 사랑이 깨진다해도
모든 것을 한꺼번에 잃는다 해도
모두들 가슴속에는 뚜렷이 있다
한마리 예쁜 고래 하나가
자 떠나자 동해 바다로
신화처럼 소리치는 고래 잡으러
자 떠나자 동해 바다로
신화처럼 소리치는 고래 잡으러
자 떠나자 고래 잡으러
신화처럼 소리치는 고래 잡으러
자 우리 떠나자 동해 바다로
신화처럼 소리치는 고래 잡으러
"""

# 토큰화
inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
inputs = inputs.to(device)

# 요약 생성 파라미터 조정
summary_ids = model.generate(
    inputs,
    max_length=50,  # 요약의 최대 길이를 조정
    length_penalty=1.0,
    num_beams=4,
    early_stopping=True,
    no_repeat_ngram_size=2  # 반복된 n-gram 방지
)

# 결과 디코딩
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 결과 출력
if summary:
    print(summary)
else:
    print("요약 생성 실패: 요약이 비어 있음")



요약 생성 실패: 요약이 비어 있음
